Reading the CSV files as DataFrames

In [156]:
import pandas as pd
inventory = pd.read_csv('/kaggle/input/sales-in-period-walmart/_5wuCXmjSp25ws6zPYB8Rg_462096727ec040c1b1a5c9ab5397acf1_Inventory.csv')
products = pd.read_csv('/kaggle/input/sales-in-period-walmart/1VLm7A0sRoSc33GdXPsFmw_bbae0bf5df6a4fd1bf43f32042ae6af1_Products.csv')
sales = pd.read_csv('/kaggle/input/sales-in-period-walmart/VoEdRv-zS0Sam9cxMydT7w_611f188c586441f1b67d32f4585abbf1_Sales.csv')

# # **Points of Interest**
* Highest Selling Product across various stores (3 Top Products per Store by Sales).
* Highest Selling Product across various Neighborhood (3 Top Products per Neighborhood by Sales).
* Top Walmart Stores by number of sales.
* Top Walmart Stores by Profit.
* Top neighborhoods by Spend.
* Products with the highest margins (Descending Order).
* Top suppliers.
* Number of stores per Neighborhood.

In [157]:
inventory.head()

,ProductId,StoreId,StoreName,Address,neighborhood,QuantityAvailable
0,1,85123,National Stores,9 Springview Point,Bolton Hill,11
1,2,71053,Family Dollar,5434 Daystar Circle,Ashburton,1
2,3,84406,BJ's Wholesale Club,3 Darwin Drive,Morrell Park,11
3,4,84029,Ocean State Job Lot,684 Bunting Lane,Fells Point,1
4,5,37444,Ollie's Bargain Outlet,50162 John Wall Drive,Charles Village,3


In [158]:
products.head()

,ProductId,ProductName,Supplier,ProductCost
0,1.0,Chocolate Bar - Smarties,National Stores,1.25
1,2.0,Pepper - Red Bell,Family Dollar,2.99
2,3.0,Chickensplit Half,BJ's Wholesale Club,0.17
3,4.0,Zucchini - Green,Ocean State Job Lot,5.28
4,5.0,"Cod - Salted, Boneless",Ollie's Bargain Outlet,0.78


In [159]:
sales.head()

,SalesId,StoreId,ProductId,Date,UnitPrice,Quantity
0,82319,22726,590,2019-12-02,0.0525,93
1,15022,21754,390,2017-11-19,5.1100,28
2,11624,71053,883,2020-07-13,7.3675,33
3,63101,22914,658,2019-05-12,2.0825,76
4,29702,22623,632,2020-07-20,0.6475,8


**Check unique_counts of the DFs.**

In [160]:
nrows_inventory = inventory.nunique()
rows_inventory = len(inventory)
nrows_products = products.nunique()
rows_products = len(products)
nrows_sales = sales.nunique()
rows_sales = len(sales)
print(f"inventory_rows \n {nrows_inventory} \n inventory_length \t {rows_inventory} \n products_rows \n {nrows_products} \n products_length \t {rows_products} \n sales_rows \n {nrows_sales} \n sales_length \t {rows_sales}")

inventory_rows 
 ProductId            1000
StoreId                34
StoreName              34
Address               999
neighborhood           32
QuantityAvailable      12
dtype: int64 
 inventory_length 	 1000 
 products_rows 
 ProductId      1000
ProductName     826
Supplier         34
ProductCost     407
dtype: int64 
 products_length 	 1001 
 sales_rows 
 SalesId      73389
StoreId         34
ProductId     1000
Date          1460
UnitPrice      407
Quantity       100
dtype: int64 
 sales_length 	 200000


**Creating Master Dataset**

In [161]:
sales['TotalAmount'] = sales['UnitPrice']*sales['Quantity']
products_sales = sales.merge(products, on = 'ProductId', how = 'inner')
master_df = products_sales.merge(inventory, on = ['StoreId', 'ProductId'], how = 'inner')
master_df.head()

,SalesId,StoreId,ProductId,Date,UnitPrice,Quantity,TotalAmount,ProductName,Supplier,ProductCost,StoreName,Address,neighborhood,QuantityAvailable
0,53757,21883,590,2020-07-12,0.0525,36,1.8900,Wine - Wyndham Estate Bin 777,Bi-Mart,0.03,Bi-Mart,718 Merchant Center,Washington Village,12
1,69487,21883,590,2020-10-06,0.0525,14,0.7350,Wine - Wyndham Estate Bin 777,Bi-Mart,0.03,Bi-Mart,718 Merchant Center,Washington Village,12
2,39033,21883,590,2017-04-29,0.0525,29,1.5225,Wine - Wyndham Estate Bin 777,Bi-Mart,0.03,Bi-Mart,718 Merchant Center,Washington Village,12
3,58868,21883,590,2019-01-16,0.0525,42,2.2050,Wine - Wyndham Estate Bin 777,Bi-Mart,0.03,Bi-Mart,718 Merchant Center,Washington Village,12
4,84594,21883,590,2017-08-11,0.0525,19,0.9975,Wine - Wyndham Estate Bin 777,Bi-Mart,0.03,Bi-Mart,718 Merchant Center,Washington Village,12


# Highest Selling Product across various stores (3 Top Products per Store by Sales).

In [162]:
SalesPerStorePerProduct = master_df.groupby(['StoreName', 'ProductName']).agg({'Quantity': sum}).reset_index().sort_values(by = ['StoreName','Quantity'], ascending = [True, False])
SalesPerStorePerProduct['Rank'] = SalesPerStorePerProduct.groupby('StoreName')['Quantity'].rank(method = 'first', ascending = False)
Top3SalesPerStorePerProduct = SalesPerStorePerProduct.loc[SalesPerStorePerProduct['Rank']<4].reset_index(drop = True)
Top3SalesPerStorePerProduct[['StoreName', 'ProductName', 'Quantity']].head()

,StoreName,ProductName,Quantity
0,BJ's Wholesale Club,Muffin - Bran Ind Wrpd,688
1,BJ's Wholesale Club,Cheese - Fontina,633
2,BJ's Wholesale Club,White Baguette,557
3,Bargain Hunt,Lemonade - Pineapple Passion,609
4,Bargain Hunt,Muffin - Banana Nut Individual,449


# Highest Selling Product across various Neighborhood (3 Top Products per Neighborhood by Sales).

In [163]:
TotalSalesPerNeighborhood = master_df.groupby(['neighborhood','ProductName']).agg({'Quantity': sum}).reset_index().sort_values(by = ['neighborhood', 'Quantity'], ascending = [True, False])
TotalSalesPerNeighborhood['Rank'] = TotalSalesPerNeighborhood.groupby('neighborhood')['Quantity'].rank(method = 'first', ascending = False)
Top3TotalSalesPerNeighborhood = TotalSalesPerNeighborhood.loc[TotalSalesPerNeighborhood['Rank']<4].reset_index(drop = True)
Top3TotalSalesPerNeighborhood[['neighborhood', 'ProductName', 'Quantity']].head()

,neighborhood,ProductName,Quantity
0,Ashburton,Chocolate Bar - Coffee Crisp,642
1,Ashburton,Crab Brie In Phyllo,638
2,Ashburton,Fireball Whisky,629
3,Baltimore Highlands,Nantucket Orange Juice,559
4,Baltimore Highlands,Sauce - Fish 25 Ozf Bottle,465


# Top Walmart Stores by number of sales.

In [164]:
TotalSalesPerStore = master_df.groupby('StoreName').agg({'Quantity': sum}).sort_values(by = 'Quantity', ascending = False).reset_index()
TotalSalesPerStore.head()

,StoreName,Quantity
0,Ben Franklin,13776
1,Family Dollar,12736
2,Shopko,11941
3,T.J. Maxx,11524
4,Fred's,11197


# Top Walmart Stores by Profit.

In [165]:
master_df['CPofSoldItems'] = master_df['ProductCost']*master_df['Quantity']
master_df['ProfitPerSale'] = master_df['TotalAmount'] - master_df['CPofSoldItems']
StoresByProfit = master_df.groupby('StoreName').agg({'ProfitPerSale': sum}).sort_values(by = 'ProfitPerSale', ascending = False).reset_index().round({'ProfitPerSale': 2})
StoresByProfit = StoresByProfit.rename(columns = {'ProfitPerSale': 'Profit'})
StoresByProfit.head()

,StoreName,Profit
0,Ben Franklin,24307.64
1,Renys,19980.26
2,Burlington Coat Factory,18011.60
3,Shopko,17773.24
4,Family Dollar,17235.40


# Top neighborhoods by Spend.

In [166]:
TopNeighborhoodsBySpend = master_df.groupby('neighborhood').agg({'TotalAmount': sum}).sort_values(by = 'TotalAmount', ascending = False).round({'TotalAmount': 2}).reset_index()
TopNeighborhoodsBySpend.head()

,neighborhood,TotalAmount
0,Bridgeview/Greenlawn,56717.83
1,Charles Village,49457.22
2,Port Covington,46620.61
3,Washington Village,43923.81
4,Upton,42027.07


# Products with the highest profit percentage.

In [239]:
pd.options.mode.chained_assignment = None
master_df['Profit'] = master_df['UnitPrice'] - master_df['ProductCost']
dropping_duplicates = master_df.drop_duplicates(subset = ['ProductName', 'Profit', 'ProductCost'])
profit_pc = dropping_duplicates[['ProductName', 'Profit', 'ProductCost']]
profit_pc['ProfitPercentage'] = 100*(profit_pc['Profit']/profit_pc['ProductCost'])
profit_pc = profit_pc.sort_values(by = 'ProfitPercentage', ascending = False)[['ProductName', 'ProfitPercentage']].reset_index(drop = True)
profit_pc.head()

,ProductName,ProfitPercentage
0,Pail With Metal Handle 16l White,75.0
1,Banana Turning,75.0
2,Rice Paper,75.0
3,"Crab - Back Fin Meat, Canned",75.0
4,Wine - Riesling Dr. Pauly,75.0


# Top Suppliers by Number of Products & Percentage

In [240]:
top_suppliers = products.groupby('Supplier').agg({'Supplier': 'count'}).rename(columns = {'Supplier': 'Count'}).sort_values(by = 'Count', ascending = False).reset_index()
total_supp = len(products['Supplier'])
top_suppliers['percentage'] = 100*(top_suppliers['Count']/total_supp)
top_suppliers.round({'percentage': 2}).head()

,Supplier,Count,percentage
0,Ben Franklin,46,4.6
1,Shopko,39,3.9
2,Family Dollar,38,3.8
3,T.J. Maxx,38,3.8
4,Walmart,36,3.6


# Number of stores per Neighborhood

In [238]:
StorePerNeighbor = inventory.groupby('neighborhood').agg({'StoreId': pd.Series.nunique}).rename(columns = {'StoreId': 'Stores'}).sort_values(by = 'Stores', ascending = False).reset_index()
StorePerNeighbor.head()

,neighborhood,Stores
0,Washington Village,2
1,Charles Village,2
2,Hampden,1
3,Upton,1
4,Seton Business Park,1
